In [1]:
import numpy as np
from pymatgen.core import Structure
from megnet.models import MEGNetModel
from megnet.data.crystal import CrystalGraph
import json

2023-11-27 14:20:24.771174: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-27 14:20:24.774541: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-27 14:20:24.817752: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 14:20:24.817790: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 14:20:24.817818: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [3]:
with open("/root/public/bandgap_json/DFT_bandgaptrain1.json", "r") as f:
    data_train = json.load(f)

In [4]:
structures = []
DFT_gaps = []

for item in data_train:
    structure = Structure.from_dict(item['structure'])
    gap = item['bandgap']
    structures.append(structure)
    DFT_gaps.append(gap)

In [5]:
structures[0]

Structure Summary
Lattice
    abc : 18.529572012331908 6.521252982657742 6.245166844454914
 angles : 89.99999038110161 89.99998025920198 90.000004174406
 volume : 754.6411478921391
      A : 18.529572012330846 1.24694756e-08 6.2709044361e-06
      B : -4.795086209e-07 6.521252982657631 1.1054893964e-06
      C : 3.81897325e-08 -1.02393447e-08 6.245166844454914
    pbc : True True True
PeriodicSite: H (4.345, 3.261, 3.123) [0.2345, 0.5, 0.5]
PeriodicSite: H (10.52, 3.261, 3.123) [0.5678, 0.5, 0.5]
PeriodicSite: H (16.7, 3.261, 3.123) [0.9012, 0.5, 0.5]
PeriodicSite: H (3.257, 3.261, 1.132) [0.1758, 0.5, 0.1812]
PeriodicSite: H (9.433, 3.261, 1.132) [0.5091, 0.5, 0.1812]
PeriodicSite: H (15.61, 3.261, 1.132) [0.8424, 0.5, 0.1812]
PeriodicSite: H (1.646, 3.261, 1.823) [0.08884, 0.5, 0.2919]
PeriodicSite: H (7.823, 3.261, 1.823) [0.4222, 0.5, 0.2919]
PeriodicSite: H (14.0, 3.261, 1.823) [0.7555, 0.5, 0.2919]
PeriodicSite: H (1.646, 3.261, 4.422) [0.08884, 0.5, 0.7081]
PeriodicSite: H (7.82

In [6]:
nfeat_bond = 10
r_cutoff = 8
gaussian_centers = np.linspace(0, r_cutoff+1, nfeat_bond)
gaussian_width = 0.5
graph_converter = CrystalGraph(cutoff=r_cutoff)
model = MEGNetModel(graph_converter=graph_converter, centers=gaussian_centers, width=gaussian_width)

2023-11-27 14:22:05.426357: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-27 14:22:05.457495: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/opt/tefscloud/miniconda3/lib/python3.9/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseede

In [8]:
# Model training
# Here, `structures` is a list of pymatgen Structure objects.
# `targets` is a corresponding list of properties.
model.train(structures, DFT_gaps, epochs=150)

Epoch 1/150
1/1 [==============================] - 22s 22s/step - loss: 3.7811
Epoch 2/150
1/1 [==============================] - 2s 2s/step - loss: 3.6583
Epoch 3/150
1/1 [==============================] - 2s 2s/step - loss: 3.5205
Epoch 4/150
1/1 [==============================] - 2s 2s/step - loss: 3.3559
Epoch 5/150
1/1 [==============================] - 2s 2s/step - loss: 3.1568
Epoch 6/150
1/1 [==============================] - 2s 2s/step - loss: 2.9152
Epoch 7/150
1/1 [==============================] - 2s 2s/step - loss: 2.6218
Epoch 8/150
1/1 [==============================] - 2s 2s/step - loss: 2.2670
Epoch 9/150
1/1 [==============================] - 2s 2s/step - loss: 1.8424
Epoch 10/150
1/1 [==============================] - 2s 2s/step - loss: 1.3481
Epoch 11/150
1/1 [==============================] - 2s 2s/step - loss: 0.8095
Epoch 12/150
1/1 [==============================] - 2s 2s/step - loss: 0.3275
Epoch 13/150
1/1 [==============================] - 2s 2s/step - loss: 

In [9]:
# 使用模型预测能带差
predictions = []
for structure in structures:
    prediction = model.predict_structure(structure)
    predictions.append(prediction)

In [11]:
# 计算均方根误差、平均绝对误差和R²,# 计算MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  

rmse = np.sqrt(mean_squared_error(DFT_gaps, predictions))
mae = mean_absolute_error(DFT_gaps, predictions)
r2 = r2_score(DFT_gaps, predictions)

In [12]:
print(r2)
print(mae)
print(rmse)

0.30509469513611487
0.2608312355135002
0.3494036201347057


In [13]:
#模型无法直接预测带隙值